In [2]:
import pandas as pd
import numpy as np
import pickle

from transformers  import  AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import util, SentenceTransformer

In [3]:
pd.set_option('max_colwidth', 200)
df = pd.read_csv('datasets/final_sample.csv')
df.info()
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93191 entries, 0 to 93190
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ParentId      93191 non-null  int64 
 1   Score         93191 non-null  int64 
 2   Response      93191 non-null  object
 3   Title         93191 non-null  object
 4   Body          93191 non-null  object
 5   title_body    93191 non-null  object
 6   title_answer  93191 non-null  object
dtypes: int64(2), object(5)
memory usage: 5.0+ MB


,ParentId,Score,Response,Title,Body,title_body,title_answer
72707,18546338,6,"You split on all spaces, then rejoin the pairs: words = inputstr.split() secondwords = iter(words) next(secondwords) output = [' '.join((first, second)) for first, second in zip(words, secondwords...",Split strings by 2nd space,"Input : ""The boy is running on the train"" Output expected: [""The boy"", ""boy is"", ""is running"", ""running on"", ""on the"", ""the train""] What is the easiest solution to achieve this in python.","Split strings by 2nd space Input : ""The boy is running on the train"" Output expected: [""The boy"", ""boy is"", ""is running"", ""running on"", ""on the"", ""the train""] What is the easiest solution to achie...","Split strings by 2nd space You split on all spaces, then rejoin the pairs: words = inputstr.split() secondwords = iter(words) next(secondwords) output = [' '.join((first, second)) for first, secon..."
83115,25517791,6,This amounts to the top-down (recursive) approach vs. the bottom-up (iterative) approach for dynamic programming. Since you know that for input n you need all values of stairs(p) for 0 <= p <= n. ...,Replacing recursion with while loop (stair climbing puzzle): Python,"I'm practicing replacing recursion with while loops, and I'm stuck on the following problem. How many ways can you go up a staircase of length n if you can only take the stairs 1 or 2 at a time? T...","Replacing recursion with while loop (stair climbing puzzle): Python I'm practicing replacing recursion with while loops, and I'm stuck on the following problem. How many ways can you go up a stair...",Replacing recursion with while loop (stair climbing puzzle): Python This amounts to the top-down (recursive) approach vs. the bottom-up (iterative) approach for dynamic programming. Since you know...
82155,24562981,13,Here is an example of batch request in Java where I get all the threads using threads ids. This can be easily adapted for your need. BatchRequest b = service.batch(); //callback function. (Can als...,Bulk-fetching emails in the new Gmail API,I'm using the python version of the newly released Gmail API by Google. The following call returns just a list of message ids: service.users().messages().list(userId = 'me').execute() But then I j...,Bulk-fetching emails in the new Gmail API I'm using the python version of the newly released Gmail API by Google. The following call returns just a list of message ids: service.users().messages()....,Bulk-fetching emails in the new Gmail API Here is an example of batch request in Java where I get all the threads using threads ids. This can be easily adapted for your need. BatchRequest b = serv...
32183,5094919,7,If you want to post binary data use the base64 encoding. http://docs.python.org/library/base64.html,Binary Data To Unicode,Among all the encodings available here http://docs.python.org/library/codecs.html which one is the one I should use for decoding binary data into unicode without it becoming corrupted when I encod...,Binary Data To Unicode Among all the encodings available here http://docs.python.org/library/codecs.html which one is the one I should use for decoding binary data into unicode without it becoming...,Binary Data To Unicode If you want to post binary data use the base64 encoding. http://docs.python.org/library/base64.html
57529,12234788,11,"Don't use eval(), use setattr() and getattr() instead: setattr(self, property, getattr(self, property) + amount)",Use eval to set unknown property,"I have an User class, that has multiple properties inside it, and I also have addPoint method for User class, which allows user to input points into any of it's properties. However, there can be u...","Use eval to set unknown property I have an User class, that has multiple properties inside it, and I also have addPoint method for User class, which allows user to input points into any of it's pr...","Use eval to set unknown property Don't use eval(), use setattr() and getattr() instead: se

In [4]:
from transformers import BertConfig
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

c:\Users\adkwo\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
try:
    embed_file = open('embed_file.obj', 'rb')
    title_embeddings = pickle.load(embed_file)
    embed_file.close()
except:
    title_embeddings = embed_model.encode(df['title_body'], convert_to_tensor=True).cpu()
    embed_file = open('embed_file.obj', 'wb') 
    pickle.dump(title_embeddings, embed_file)
    embed_file.close()

In [6]:
def getSimiliar(input, k):
    input_embedding = embed_model.encode(input, convert_to_tensor=True).cpu()
    cos_score = (util.cos_sim(input_embedding, title_embeddings)).numpy()[0]

    indices = sorted(range(len(cos_score)), key=lambda x: cos_score[x])[-k:]
    indices.reverse()
 
    res = df.iloc[indices][['ParentId', 'title_body', 'Response', 'Score']]
    res['CosSim'] = cos_score[indices]

    text = ''
    for i in res['Response'].to_list():
        text += ' '+ i
    text = text.strip()
    
    return res, text

In [15]:
# Function for generating text output given a corpus of specific answers retrieved from the dataset.
# Currently uses pre-trained t5-base model for tokenizer and text generator

def generateOutput(prompt, context):
    input = f'summarize: {prompt} context: {context}'
    encoded_input = tokenizer(input,
                              truncation=True,
                              return_tensors='pt')
    output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask,
                            max_length=500)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [16]:
# display random samples of dataset entries to curate a user query
display(df['title_body'].sample(5))

53382    Using/importing Beautiful Soup 4 without installation As the Beautiful Soup documentation says: If all else fails, the license for Beautiful Soup allows you to package the entire library with your...
32138    How do I prevent a C shared library to print on stdout in python? I work with a python lib that imports a C shared library that prints on stdout. I want a clean output in order to use it with pipe...
21611    Equivalent to GetTickCount() on Linux I'm looking for an equivalent to GetTickCount() on Linux. Presently I am using Python's time.time() which presumably calls through to gettimeofday(). My conce...
71342    How to get an UTC date string in Python? I am trying to get utc date string as "YYYYMMDD" For now I do the following, nowTime = time.gmtime(); nowDate = date(nowTime.tm_year, nowTime.tm_mon, nowTi...
42363                                                                 How to set the sharing rights of a folder in Plone? I want to set sharing rights of many folde

In [17]:
# enter a query and generate a response
query = 'How to apply bitwise operator to compare a list of objects'
response, text = getSimiliar(query, 5)

In [18]:
# compate dataset answer to generated response
output = generateOutput(query, text)
display(response)
print(f'Full text: {text} \n')
print(f'Generated output: {output}')

,ParentId,title_body,Response,Score,CosSim
92838,39153200,"How to apply bitwise operator to compare a list of objects Suppose I have a long list of objects (say, a list of numpy matrices of bool elements) foo = [a, b, c], that I want to compare with some ...","Using the functional method in operator in combination with reduce: >>> import operator, functools >>> functools.reduce(operator.or_, [1,2,3]) 3",8,0.811140
83370,25796158,"bitwise operations between elements in a list I have a of list of bitwise elements, e.g. [1,1,1], and I want to do a bitwise OR operation between every element in the list. So, e.g. for [1,1,1] do...","You can use reduce with operator.ior: >>> from operator import ior >>> lst = [1, 17, 1] >>> reduce(ior, lst) 17 And as suggested by @DSM in comments the numpy equivalent will be: >>> import numpy ...",8,0.613057
89208,32406128,"Bitwise-OR the elements of a list in python using lambda Can I use the lambda function this way to bitwise-OR all the elements in the list? lst = [1, 1, 1] f = lambda x: x | b for b in lst When I ...","You want reduce: f = reduce(lambda x, y: x | y, lst) reduce accepts a binary function and an iterable, and applies the operator between all elements starting from the first pair. Note: in Python 3...",9,0.585091
24803,3555375,"Given a list and a bitmask, how do I return the values at the indices that are True? I start with the following list s and bitmask b: s = ['baa', 'baa', 'black', 'sheep', 'have', 'you', 'any', 'wo...","Python 3.1 itertools.compress (or Python 2.7's if you haven't upgraded yet) does exactly that (the list comprehension is a real close second): import itertools filtered = itertools.compress(s, b) ...",15,0.569773
24802,3555375,"Given a list and a bitmask, how do I return the values at the indices that are True? I start with the following list s and bitmask b: s = ['baa', 'baa', 'black', 'sheep', 'have', 'you', 'any', 'wo...","You can use list comprehensions: newList = [word for (word, mask) in zip(s,b) if mask] # Note: Could also use 'if mask == blah', if mask is not a boolean-compatible type. This first takes the orig...",7,0.569773


Full text: Using the functional method in operator in combination with reduce: >>> import operator, functools >>> functools.reduce(operator.or_, [1,2,3]) 3 You can use reduce with operator.ior: >>> from operator import ior >>> lst = [1, 17, 1] >>> reduce(ior, lst) 17 And as suggested by @DSM in comments the numpy equivalent will be: >>> import numpy as np >>> arr = np.array(lst) >>> np.bitwise_or.reduce(arr) 17 You want reduce: f = reduce(lambda x, y: x | y, lst) reduce accepts a binary function and an iterable, and applies the operator between all elements starting from the first pair. Note: in Python 3 it moves to the functools module. You can also use the or_ function from the operator module instead of writing the lambda yourself: from operator import or_ f = reduce(or_, lst) Python 3.1 itertools.compress (or Python 2.7's if you haven't upgraded yet) does exactly that (the list comprehension is a real close second): import itertools filtered = itertools.compress(s, b) Note that thi